I used this document to profile my ~/.bashrc file.

# Requirements:

    * Python 3
    * daemontools(tai64n, tai64nlocal)
    
# Conclusion:

``thefuck`` was causing a second plus holdup, and I had to omment it out.

# Imports

In [9]:
from datetime import datetime
import itertools
from operator import itemgetter

# Get the data

In [10]:
time_txt = !sh -x ~/.bashrc 2>&1 | tai64n | tai64nlocal

Here's what it looks like:

In [11]:
time_txt[:10]

['2016-11-29 05:54:10.728393500 + [[ hxB != *i* ]]',
 '2016-11-29 05:54:10.728394500 + return',
 "2016-11-29 05:54:10.728474500 /home/bguinta/.bashrc: line 2: return: can only `return' from a function or sourced script",
 '2016-11-29 05:54:10.728777500 + [[ -f /home/bguinta/.bashrc.local ]]',
 '2016-11-29 05:54:10.728797500 + . /home/bguinta/.bashrc.local',
 '2016-11-29 05:54:10.728856500 ++ [[ -f /home/bguinta/.extend.bashrc ]]',
 '2016-11-29 05:54:10.728885500 ++ . /home/bguinta/.extend.bashrc',
 '2016-11-29 05:54:10.729013500 +++ case ${TERM} in',
 '2016-11-29 05:54:10.729064500 +++ PROMPT_COMMAND=\'echo -ne "\\033]0;${USER}@${HOSTNAME%%.*}:${PWD/#$HOME/~}\\007"\'',
 '2016-11-29 05:54:10.729108500 +++ use_color=true']

In [12]:
# Taken from boltons
def windowed_iter(src, size):
    """Returns tuples with length *size* which represent a sliding
    window over iterable *src*.

    >>> list(windowed_iter(range(7), 3))
    [(0, 1, 2), (1, 2, 3), (2, 3, 4), (3, 4, 5), (4, 5, 6)]

    If the iterable is too short to make a window of length *size*,
    then no window tuples are returned.

    >>> list(windowed_iter(range(3), 5))
    []
    """
    # TODO: lists? (for consistency)
    tees = itertools.tee(src, size)
    try:
        for i, t in enumerate(tees):
            for _ in range(i):
                next(t)
    except StopIteration:
        return zip([])
    return zip(*tees)

In [13]:
def extract_timestring(text_list):
    """Grab Only relevent time data(Hours Minutes Seconds Microseconds)"""
    for line in time_txt:
        _, time, *cmd = line.split()
        yield time
def str_to_datetime(time_list):
    """Chop the bogus nano seconds art of the string. Convert to datetime"""
    for time_string in time_list:
        yield datetime.strptime(time_string[:-3], "%H:%M:%S.%f")
def generate_deltas(datetime_list):    
    """Return the time taken between every line in the rcfile"""
    for  date1, date2 in windowed_iter(datetimes, 2):
        yield (date2 - date1)
    

# Processing

In [14]:
datetimes = str_to_datetime(extract_timestring(time_txt))
deltas = generate_deltas(datetimes)
delta_pair = enumerate(deltas)
delta_descending = sorted(delta_pair, key=itemgetter(1), reverse=True)

# Formatting

In [15]:
format_str = "{:<9} | {}"
print(format_str.format("Time", "Command"))
for i, delta in delta_descending:
    # Remove tai64nlocal date and formatting
    cmd_text = " ".join(time_txt[i].split(" ")[2:]).lstrip("+ ")
    print(format_str.format(delta.total_seconds(), cmd_text))
    # Ignore lines that take less than 100 microseconds
    if delta.microseconds <= 100:
        break

Time      | Command
0.99109   | thefuck --alias
0.004467  | dircolors -b /home/bguinta/.dir_colors
0.002557  | grep --colour=auto '_z --add'
0.001847  | xhost +local:root
0.001226  | readlink -f /home/bguinta/.bashrc
0.00091   | [[ -f /home/bguinta/.dir_colors ]]
0.000786  | '[' -d /home/bguinta/.z ']'
0.000759  | '[' -r /usr/share/bash-completion/bash_completion ']'
0.000746  | dirname /home/bguinta/dotfiles/bashrc
0.000467  | [[ -f /home/bguinta/.dir_colors ]]
0.000389  | type thefuck
0.000303  | /home/bguinta/.bashrc: line 2: return: can only `return' from a function or sourced script
0.000231  | eval 'LS_COLORS='\''rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=01;05;37;41:mi=01;05;37;41:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31: